# **Kaggle에서 데이터 가져오기**




In [ ]:
!pip install kaggle
from google.colab import files
files.upload()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Saving kaggle.json to kaggle (1).json


{'kaggle.json': b'{"username":"jangjiyun","key":"e98e76a302255cea3594dcd4ddad1767"}'}

In [ ]:
ls -1ha kaggle.json

kaggle.json


In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Permission Warning 이 일어나지 않도록 
!chmod 600 ~/.kaggle/kaggle.json
# 본인이 참가한 모든 대회 보기 
!kaggle competitions list

ref                                                                                 deadline             category            reward  teamCount  userHasEntered  
----------------------------------------------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
https://www.kaggle.com/competitions/asl-signs                                       2023-05-01 23:59:00  Research          $100,000        411           False  
https://www.kaggle.com/competitions/tlvmc-parkinsons-freezing-gait-prediction       2023-06-08 23:59:00  Research          $100,000          0           False  
https://www.kaggle.com/competitions/godaddy-microbusiness-density-forecasting       2023-03-14 23:59:00  Featured           $60,000       3419           False  
https://www.kaggle.com/competitions/amp-parkinsons-disease-progression-prediction   2023-05-18 23:59:00  Featured           $60,000        401           False  
https://www.kaggle.com/competition

In [ ]:
!kaggle competitions download -c aikuthon-2023-01

100% 2.54G/2.54G [02:02<00:00, 26.2MB/s]
100% 2.54G/2.54G [02:02<00:00, 22.2MB/s]


In [ ]:
!ls

aikuthon-2023-01.zip  kaggle.json  sample_data


In [ ]:
!unzip aikuthon-2023-01.zip

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  inflating: data/data/train/imgs/train_2702.png  
  inflating: data/data/train/imgs/train_2703.png  
  inflating: data/data/train/imgs/train_2704.png  
  inflating: data/data/train/imgs/train_2705.png  
  inflating: data/data/train/imgs/train_2706.png  
  inflating: data/data/train/imgs/train_2707.png  
  inflating: data/data/train/imgs/train_2708.png  
  inflating: data/data/train/imgs/train_2709.png  
  inflating: data/data/train/imgs/train_2710.png  
  inflating: data/data/train/imgs/train_2711.png  
  inflating: data/data/train/imgs/train_2712.png  
  inflating: data/data/train/imgs/train_2713.png  
  inflating: data/data/train/imgs/train_2714.png  
  inflating: data/data/train/imgs/train_2715.png  
  inflating: data/data/train/imgs/train_2716.png  
  inflating: data/data/train/imgs/train_2717.png  
  inflating: data/data/train/imgs/train_2718.png  
  inflating: data/data/train/imgs/train_2719.png  
  inflating: data/data/train/imgs/train_2720.p

# **Data Loader**

In [ ]:
!nvidia-smi

Fri Apr 28 11:20:27 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   52C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install pydicom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 41.8 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import random
from sklearn.model_selection import train_test_split
# from IPython.display import Image
import pydicom
import torchvision
from torchvision import transforms, datasets, models
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from PIL import Image

from tqdm.auto import tqdm

import matplotlib.pyplot as plt
import matplotlib.patches as patches


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [ ]:
cd /content/drive/MyDrive/Colab_Notebooks/github/AIKU_deepintodeep/AIKUTHON

/content/drive/MyDrive/Colab_Notebooks/github/AIKU_deepintodeep/AIKUTHON


In [ ]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [ ]:
img_path = 'aikuthon_2023_01/data/data/train/imgs/'
mask_path = 'aikuthon_2023_01/data/data/train/masks/'
test_path = 'aikuthon_2023_01/data/data/test/imgs/'
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(device)
img_transforms = transforms.Compose([
        transforms.ToTensor(),
])
mask_transforms = transforms.Compose([
    transforms.PILToTensor(),
])

cuda


# **Data Check & Data Loader**

In [ ]:
!ls

 2023_AIKUTHON.ipynb   finetuning-image-segmentation   result.csv
 aikuthon_2023_01     'kaggle (1).json'		       wandb
 checkpoint.pt	       kaggle.json


In [ ]:
imgs = list(sorted(os.listdir(img_path)))
masks = list(sorted(os.listdir(mask_path)))
train_ims, val_ims, train_mask, val_mask = train_test_split(imgs, masks)
print(len(train_ims), len(train_mask), len(val_ims), len(val_mask))
print()

2887 2887 963 963



In [ ]:
print(train_ims[0], train_mask[0])
print(train_ims[1], train_mask[1])


train_2330.png train_2330.png
train_3450.png train_3450.png


In [ ]:
#train_ims = list(sorted(os.listdir(img_path)))
#train_mask = list(sorted(os.listdir(mask_path)))
test_ims = list(sorted(os.listdir(test_path)))
#print(len(train_ims))
#print(len(train_mask))
#print(train_ims)

In [ ]:
class SegmentationDataSet(torch.utils.data.Dataset):
    def __init__(self, img_dir, mask_dir, image_list, mask_list, img_transforms, mask_transforms):
        self.img_transforms = img_transforms
        self.mask_transforms = mask_transforms
        self.imgs = image_list
        self.masks = mask_list
        self.img_dir, self.mask_dir = img_dir, mask_dir


    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        idx = idx + 1
        str_idx = f'{idx:04d}'
        file_image = 'train_'+ str_idx + '.png'
        file_mask = 'train_'+ str_idx + '.png'
        
        img_path = os.path.join(self.img_dir, file_image)
        mask_path = os.path.join(self.mask_dir, file_mask)
        
        img = Image.open(img_path).convert("RGB")
        masks = Image.open(mask_path).convert("RGB")
        img = self.img_transforms(img)
        masks = self.mask_transforms(masks)
        masks = masks[0, :, :]
        pad = (0, 0, 4, 4)
        images_pad = F.pad(img, pad, "constant", 0)
        return images_pad, masks
        
def collate_fn(batch) : return tuple(zip(*batch))

class SegmentationTestDataSet(torch.utils.data.Dataset):
    def __init__(self, img_dir, image_list, transforms):
        self.transforms = transforms
        self.imgs = image_list
        self.img_dir = img_dir


    def __len__(self):
        return len(self.imgs)

    def __getitem__(self, idx):
        idx = idx + 1
        str_idx = f'{idx:04d}'
        file_image = 'test_'+ str_idx + '.png'
        
        img_path = os.path.join(self.img_dir, file_image)
        
        img = Image.open(img_path).convert("RGB")
        img = self.transforms(img)
        pad = (0, 0, 4, 4)
        images_pad = F.pad(img, pad, "constant", 0)
        return images_pad, 'test_'+str_idx

In [ ]:
# DataLoader 확인

train_data = SegmentationDataSet(img_path, mask_path, train_ims, train_mask, img_transforms=img_transforms, mask_transforms=mask_transforms)
train_dataloader = torch.utils.data.DataLoader(train_data, batch_size = 4, shuffle = True, drop_last = True)

val_data = SegmentationDataSet(img_path, mask_path, val_ims, val_mask, img_transforms=img_transforms, mask_transforms=mask_transforms)
val_dataloader = torch.utils.data.DataLoader(val_data, batch_size = 4, shuffle = True, drop_last = True)


test_data = SegmentationTestDataSet(test_path,  test_ims, transforms=img_transforms)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size = 1, shuffle = False)
for img, file in (test_dataloader):
    print(img.shape, file[0])
    break

#for i, (inputs, labels) in enumerate(train_dataloader):
 #   output = torch.stack([inputs[0], inputs[1], inputs[2], inputs[3], inputs[4]], dim=0)
  #  print(output.shape)
   # break

for i, (img, masks) in enumerate(train_dataloader):
    print(img.size())
    print(masks.shape)
    print(masks.squeeze())
    break

torch.Size([1, 3, 608, 800]) test_0001
torch.Size([4, 3, 608, 800])
torch.Size([4, 600, 800])
tensor([[[ 9,  9,  9,  ...,  9,  9,  9],
         [ 0,  0,  0,  ...,  9,  9,  9],
         [ 0,  0,  0,  ...,  9,  9,  9],
         ...,
         [10, 10, 10,  ..., 10, 10, 10],
         [10, 10, 10,  ..., 10, 10, 10],
         [10, 10, 10,  ..., 10, 10, 10]],

        [[ 9,  9,  9,  ...,  0,  0,  0],
         [ 9,  9,  9,  ...,  0,  0,  0],
         [ 9,  9,  9,  ...,  0,  0,  0],
         ...,
         [10, 10, 10,  ..., 10, 10, 10],
         [10, 10, 10,  ..., 10, 10, 10],
         [10, 10, 10,  ..., 10, 10, 10]],

        [[ 9,  9,  9,  ...,  0,  0,  0],
         [ 9,  9,  9,  ...,  0,  0,  0],
         [ 9,  9,  9,  ...,  0,  0,  0],
         ...,
         [10, 10, 10,  ..., 10, 10, 10],
         [10, 10, 10,  ..., 10, 10, 10],
         [10, 10, 10,  ..., 10, 10, 10]],

        [[ 0,  0,  0,  ...,  0,  0,  0],
         [ 0,  0,  0,  ...,  0,  0,  0],
         [ 0,  0,  0,  ...,  0,  0,  0

# **MODELING**

In [ ]:
!pip install segmentation-models-pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 7.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 549.1/549.1 kB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 9.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.3 MB/s eta 0:00:00
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16444 sha256=db06ae3975cbede184647a31451ac6090e6d9d43774ce4d66d6044b8cf8b9a22
  Stored in directory: /root/.cache/pip/wheels/03/3f/e9/911b1bc46869644912bda90a56bcf7b960f20b5187feea3baf
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60962 sha256=1f6c744ecd2806e2c107d806952db7846a272a52bcd7503a40f408dbf4600bd0
  Stored in directory: /root/.cache/pip/w

In [ ]:
import segmentation_models_pytorch as smp

model = smp.DeepLabV3Plus(in_channels=3,classes=13).to(device)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 141MB/s]


In [ ]:
for name, param in model.named_parameters():
    print(name)

encoder.conv1.weight
encoder.bn1.weight
encoder.bn1.bias
encoder.layer1.0.conv1.weight
encoder.layer1.0.bn1.weight
encoder.layer1.0.bn1.bias
encoder.layer1.0.conv2.weight
encoder.layer1.0.bn2.weight
encoder.layer1.0.bn2.bias
encoder.layer1.1.conv1.weight
encoder.layer1.1.bn1.weight
encoder.layer1.1.bn1.bias
encoder.layer1.1.conv2.weight
encoder.layer1.1.bn2.weight
encoder.layer1.1.bn2.bias
encoder.layer1.2.conv1.weight
encoder.layer1.2.bn1.weight
encoder.layer1.2.bn1.bias
encoder.layer1.2.conv2.weight
encoder.layer1.2.bn2.weight
encoder.layer1.2.bn2.bias
encoder.layer2.0.conv1.weight
encoder.layer2.0.bn1.weight
encoder.layer2.0.bn1.bias
encoder.layer2.0.conv2.weight
encoder.layer2.0.bn2.weight
encoder.layer2.0.bn2.bias
encoder.layer2.0.downsample.0.weight
encoder.layer2.0.downsample.1.weight
encoder.layer2.0.downsample.1.bias
encoder.layer2.1.conv1.weight
encoder.layer2.1.bn1.weight
encoder.layer2.1.bn1.bias
encoder.layer2.1.conv2.weight
encoder.layer2.1.bn2.weight
encoder.layer2.1.bn2

In [ ]:
#update_param = ['decoder.block2.1.weight', 'segmentation_head.0.weight']

In [ ]:
#for name, param in model.named_parameters():
#    if name in update_param:
#        param.requires_grad = True
#        print(name)
#    else:
#        param.requires_grad = False


In [ ]:
def DICE(pred,target):
    target = target.float()
    smooth = 1e-5
    intersect = torch.sum(pred * target)
    y_sum = torch.sum(target * target)
    z_sum = torch.sum(pred * pred)
    loss = (2 * intersect + smooth) / (z_sum + y_sum + smooth)
    return loss

In [ ]:
for image, mask in (val_dataloader):
        
              images = image.to(device)
            masks = mask.to(device)
            pred = model(images)
            pred = pred[:, :, 4:604, :]
            print(pred.shape, masks.shape)
            pred = pred.argmax(dim=1)
            #loss = DICE(pred, masks)
            prediction = pred
            dice_score = 0
            for j in range(13):
                mask_label = torch.zeros(prediction.shape)
                mask_label[prediction==j]=1
                mask_labels = []
                mask_labels.append(mask_label)
                gt_one_label = []
                gt_one = torch.zeros(masks.shape)
                gt_one[masks==j]=1

                gt_one_label.append(gt_one)
                print(mask_label.shape, gt_one.shape)
                dice_score = dice_score + DICE(mask_label.squeeze(),gt_one.squeeze())  
                
                loss = 1.0- DICE(mask_label.squeeze(),gt_one.squeeze())
                print('DICE LOSS= ', loss, loss.item())
                
            print('score : ', dice_score/13.0)
            break


torch.Size([4, 13, 600, 800]) torch.Size([4, 600, 800])
torch.Size([4, 600, 800]) torch.Size([4, 600, 800])
DICE LOSS=  tensor(0.8437) 0.8437004089355469
torch.Size([4, 600, 800]) torch.Size([4, 600, 800])
DICE LOSS=  tensor(0.7904) 0.7903516888618469
torch.Size([4, 600, 800]) torch.Size([4, 600, 800])
DICE LOSS=  tensor(0.9836) 0.983610212802887
torch.Size([4, 600, 800]) torch.Size([4, 600, 800])
DICE LOSS=  tensor(0.9716) 0.971640408039093
torch.Size([4, 600, 800]) torch.Size([4, 600, 800])
DICE LOSS=  tensor(1.) 1.0
torch.Size([4, 600, 800]) torch.Size([4, 600, 800])
DICE LOSS=  tensor(0.9916) 0.9915536046028137
torch.Size([4, 600, 800]) torch.Size([4, 600, 800])
DICE LOSS=  tensor(0.9780) 0.978009819984436
torch.Size([4, 600, 800]) torch.Size([4, 600, 800])
DICE LOSS=  tensor(0.9935) 0.9934853911399841
torch.Size([4, 600, 800]) torch.Size([4, 600, 800])
DICE LOSS=  tensor(0.9370) 0.9369843602180481
torch.Size([4, 600, 800]) torch.Size([4, 600, 800])
DICE LOSS=  tensor(0.9688) 0.968

In [ ]:
torch.cuda.empty_cache()

# **WANDB**



In [ ]:
import wandb

In [ ]:
!pip install --upgrade wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import wandb

In [ ]:
wandb.login()

AttributeError: ignored

In [ ]:
sweep_config = {
    'name' : 'AIKUTHON_test',
    'method' : 'random',
    'metric' : {
    'name': 'val_score',
    'goal': 'maximize'   
    },
    'parameters' : {
        'optimizer' : {
            'values' : ['adam', 'sgd']
        }
        'learning_rate': {
        # a flat distribution between 0 and 0.1
        'distribution': 'uniform',
        'min': 0,
        'max': 0.3
      }
    }
}

In [ ]:
import pprint

pprint.pprint(sweep_config)

# **Train**

In [ ]:
def build_optimizer(optimizer, learning_rate):
    params = [p for p in model.parameters() if p.requires_grad]
    if optimizer == "sgd":
        optimizer = optim.SGD(params, lr = learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(params, lr = learning_rate)
    return optimizer

In [ ]:
num_epochs = 10
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.02, momentum=0.9, weight_decay=0.0005)
criterion = nn.CrossEntropyLoss()

In [ ]:
def train_fn(optimizer):
    model.train()
    wandb.watch(model, criterion, log="all", log_freq = 1)

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch + 1, num_epochs))
        total_loss = 0.0
        for image, mask in tqdm(train_dataloader):
            images = image.to(device)
            masks = mask.to(device).long()
            optimizer.zero_grad()

            pred = model(images)
            pred = pred[:, :, 4:604, :]
            loss = criterion(pred, masks)
            #progress.set_description(f"Epoch {epoch + 1}/{num_epochs}, C: {losses['loss_classifier'].item():.5f}, B: {losses['loss_box_reg'].item():.5f}, O: {losses['loss_objectness'].item():.5f}, T: {loss.item():.5f}")
            # print(
            #     f"{epoch}, {i}, C: {losses['loss_classifier'].item():.5f},  "
            #     f"B: {losses['loss_box_reg'].item():.5f}, O: {losses['loss_objectness'].item():.5f}, T: {loss.item():.5f}")
            loss.backward()
            
            optimizer.step()
            total_loss += loss.item()
        
        wandb.log({"train_loss": total_loss / len(train_dataloader)}, step = epoch)
        print(f"Epoch {epoch + 1} Total loss: {total_loss / len(train_dataloader)}")

        #torch.save(model.state_dict(), 'checkpoint.pt')

In [ ]:
def val_fn():
    model.eval()
    score = 0
    length = 0
    with torch.no_grad():
        for image, mask in (val_dataloader):
                length = length +1
                images = image.to(device)
                masks = mask.to(device)
                pred = model(images)
                pred = pred[:, :, 4:604, :]
                pred = pred.argmax(dim=1)
                #loss = DICE(pred, masks)
                prediction = pred
                dice_score = 0
                for j in range(13):
                    mask_label = torch.zeros(prediction.shape)
                    mask_label[prediction==j]=1
                    mask_labels = []
                    mask_labels.append(mask_label)
                    gt_one_label = []
                    gt_one = torch.zeros(masks.shape)
                    gt_one[masks==j]=1

                    gt_one_label.append(gt_one)

                    dice_score = dice_score + DICE(mask_label.squeeze(),gt_one.squeeze())  

                score = score + dice_score/13.0
    val_score = score/length
    print(val_score)
    wandb.log({
        "val_score" : val_score
    })
    return val_score


In [ ]:
def run_sweep(config = None):
    wandb.init(config = config)
    w_config = wandb.config
    optimizer = build_optimizer(w_config.optimizer, w_config.learning_rate)
    train_fn(optimizer)
    val_fn()

In [ ]:
sweep_id = wandb.sweep(sweep_config, project = "AIKUTHON_1", entity = 'fick17')
wandb.agent(sweep_id, run_sweep, count=10)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


Create sweep with ID: 6pi8w9qt
Sweep URL: https://wandb.ai/fick17/AIKUTHON_1/sweeps/6pi8w9qt


wandb: Agent Starting Run: fk8xo7u0 with config:
wandb: 	learning_rate: 0.24406867495393075


Epoch 1/5


  0%|          | 0/721 [00:00<?, ?it/s]

...

Epoch 1/5


  0%|          | 0/721 [00:00<?, ?it/s]

wandb: Ctrl + C detected. Stopping sweep.


Error in callback <function _WandbInit._pause_backend at 0x7fccb04d93f0> (for post_run_cell):


BrokenPipeError: ignored

# **Submission**

In [ ]:
def rle_encode(mask_image):
    pixels = mask_image.flatten()
    # We avoid issues with '1' at the start or end (at the corners of
    # the original image) by setting those pixels to '0' explicitly.
    # We do not expect these to be non-zero for an accurate mask,
    # so this should not harm the score.
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] = runs[1::2] - runs[:-1:2]
    return runs

In [ ]:
### 제출 방법입니다. 대표적으로 train한 모델의 weight를 저장한 후에 test할시, 모델의 weight를 불러와서 test image를 inference 해줍니다.
model_path = 'checkpoint.pt'
test_model = model
#test_model.load_state_dict(torch.load(model_path))

###batch_size=1, shuffle=False로 해야합니다. 그래야 sample_submission에 나와있는 img_id순서와 동일하게 순서가 일치하게 됩니다.
testloader = DataLoader(test_data, batch_size = 1, shuffle = False)
test_model.eval()


out_dict = []
output = []
inputs = []

for image, file in tqdm(testloader):
    ##수도코드인지라 편한대로 짜시면 됩니다.
    images, image_name = image, file                ##images:이미지파일, image_name: 이미지 이름
    images = images.to(device)
    prediction = test_model(images)
    ### segmentation이기에 가장 확률이 높은 index만을 찾아줍니다.
    #prediction = prediction[:, :, 4:604, :]
    prediction = prediction.argmax(dim=1)
    ##prediction shape: [1,600,800]
    mask_labels = []
    
    ##제출코드 및 RLE encoding을 해주는 방법입니다.
    ##img_name[0]에는 images파일의 이름이 들어가야합니다.
    ##만약 test_1001.png파일을 불러왔다면 img_name[0]='test_1001'이 들어와야 합니다.
    for j in range(13):
        mask_label = torch.zeros(prediction.shape)
        mask_label[prediction==j]=1
            
        mask_labels.append(mask_label)
        
    for j in range(0,13):
        mask_label = mask_labels[j].squeeze().numpy()
        encode = rle_encode(mask_label)
        out_dict.append((f'{image_name[0]}_{j}', ' '.join(str(_) for _ in encode)))
    

## csv만드는 코드입니다.
import pandas as pdb

df = pdb.DataFrame(out_dict)
df.columns=['ImageId','EncodedPixels']
df=df.set_index('ImageId')

df.to_csv('result.csv')

  0%|          | 0/1150 [00:00<?, ?it/s]

In [ ]:
!kaggle competitions submit -c aikuthon-2023-01 -f result.csv -m "submit"

100% 265M/265M [00:17<00:00, 16.3MB/s]
Successfully submitted to 제 1회 AIKUTHON